In [59]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re
import datetime
import dateutil
import calendar
from datetime import datetime

In [60]:
#Creating a DataFrame with service parts
df = pd.read_csv('Service_parts.csv')

In [61]:
#Removing unwanted columns
df = df.drop(columns='Unnamed: 0')

In [62]:
#Grouping filters
WaterFilter = df.loc[df['Item Number'].str.contains('WF', flags=re.I, regex=True)]
Airfilter = df.loc[df['Item Number'].str.contains('AF|AH', flags=re.I, regex=True)]
Lubefilter = df.loc[df['Item Number'].str.contains('LF', flags=re.I, regex=True)]
Fuelfilter = df.loc[df['Item Number'].str.contains('FF|FS', flags=re.I, regex=True)]
Premium = df.loc[df['Item  Description'].str.contains('PREMIUM', flags=re.I, regex=True)]
Coolant = df.loc[df['Item  Description'].str.contains('COOLANT', flags=re.I, regex=True)]

In [63]:
#Transforming quantity to int
WaterFilter['Quantity'] = pd.to_numeric(WaterFilter['Quantity']).astype(int)
Airfilter['Quantity'] = pd.to_numeric(Airfilter['Quantity']).astype(int)
Fuelfilter['Quantity'] = pd.to_numeric(Fuelfilter['Quantity']).astype(int)
Lubefilter['Quantity'] = pd.to_numeric(Lubefilter['Quantity']).astype(int)
Coolant['Quantity'] = pd.to_numeric(Coolant['Quantity']).astype(int)
#Removing fuel water separators from Water filter category
WaterFilter = WaterFilter.loc[~WaterFilter['Item  Description'].str.contains('fuel', flags=re.I, regex=True)]

In [64]:
WaterFilter = WaterFilter.reset_index(drop=True)

In [65]:
#Extracting month and year from date column
WaterFilter['Month'] = pd.DatetimeIndex(WaterFilter['Date']).month
WaterFilter['Year'] = pd.DatetimeIndex(WaterFilter['Date']).year
WaterFilter['Month'] = WaterFilter.apply(
    lambda row: '{:%b}'.format(datetime.strptime(str(row['Month']), '%m')),
    axis=1
) 
WaterFilter['Quater']= WaterFilter['Month'].map({
    'Jan':'2',
    'Feb':'2',
    'Mar':'2',
    'Apr':'3',
    'May':'3',
    'Jun':'3',
    'Jul':'4',
    'Aug':'4',
    'Sep':'4',
    'Oct':'1',
    'Nov':'1',
    'Dec':'1'
})


In [84]:
#Filtering out major clients who have left
WaterFilter = WaterFilter.loc[~WaterFilter['Customer Name'].str.contains('libya|innscor|world food|housing finance', flags=re.I)]
WaterFilter.head()

,Date,Customer Name,Item Number,Item Description,Quantity,Unit Price,Month,Year,Quater
1,18-Dec-17,HARAMBEE COOPERATIVE SAVINGS,WF2126,DCA4 WATER FILTER,1,"4,846.00",Dec,2017,1
2,18-Dec-17,TWIGA STATIONERS & PRINTERS,WF2126,DCA4 WATER FILTER,1,"4,580.00",Dec,2017,1
3,18-Dec-17,BAYER EAST AFRICA LTD,WF2074,WATER FILTER,1,"2,131.20",Dec,2017,1
4,18-Dec-17,FIDELITY SHIELD INSURANCE CO. LTD,WF2126,DCA4 WATER FILTER,1,"3,690.00",Dec,2017,1
5,18-Dec-17,UNIVERSITY OF NAIROBI LOWER KABETE,WF2126,DCA4 WATER FILTER,1,"4,100.00",Dec,2017,1


In [67]:
#Aggregate after eliminating KCB, Libya Oil
Avg_by_quater2 = WaterFilter.loc[WaterFilter['Quater']=='3'].groupby(['Item Number', 'Year']).sum()
Avg_by_quater2

Quantity
Item Number Year          
WF2051      2018         1
WF2054      2018         7
            2019        25
WF2056      2019         8
WF2071      2018        11
            2019        63
            2020        38
WF2072      2018         1
            2019         4
            2021         6
WF2073      2018        12
            2019        59
            2020        13
WF2074      2018        33
            2019        46
            2020        19
WF2075      2018        96
            2019       114
            2020        50
WF2076      2018        93
            2019       111
            2020       145
WF2096      2018        10
            2019        28
            2020         2
WF2104      2018         4
            2020         3
WF2126      2018        53
            2019       141
            2020        33

In [95]:
Fuelfilter = df.loc[df['Item Number'].str.contains('FF|FS', flags=re.I, regex=True)]
Fuelfilter = Fuelfilter.drop(columns=['Item  Description'])

In [96]:
#Analysis for fuel filters
#Extracting month and year from date column
Fuelfilter['Month'] = pd.DatetimeIndex(Fuelfilter['Date']).month
Fuelfilter['Year'] = pd.DatetimeIndex(Fuelfilter['Date']).year
Fuelfilter['Month'] = Fuelfilter.apply(
    lambda row: '{:%b}'.format(datetime.strptime(str(row['Month']), '%m')),
    axis=1
) 
Fuelfilter['Quater']= Fuelfilter['Month'].map({
    'Jan':'2',
    'Feb':'2',
    'Mar':'2',
    'Apr':'3',
    'May':'3',
    'Jun':'3',
    'Jul':'4',
    'Aug':'4',
    'Sep':'4',
    'Oct':'1',
    'Nov':'1',
    'Dec':'1'
})
Fuelfilter

,Date,Customer Name,Item Number,Quantity,Unit Price,Month,Year,Quater
12,18-Dec-17,BAYER EAST AFRICA LTD,FF5580,1,"2,680.20",Dec,2017,1
13,18-Dec-17,BAYER EAST AFRICA LTD,FS19732,1,"6,565.50",Dec,2017,1
28,30-Jan-20,PARAMOUNT PLAZA,FF5485,2,"5,342.00",Jan,2020,2
36,3-Oct-17,CASH SALES - CUMMINS,FF5524,2,560.74,Oct,2017,1
41,3-Oct-17,MATER HOSPITAL,FF5524,2,"1,100.00",Oct,2017,1
...,...,...,...,...,...,...,...,...
35283,25-Jan-21,ASHNIL HOTELS,FF5612,2,"2,008.05",Jan,2021,2
35295,28-Jan-21,DIAMOND TRUST BANK KENYA LIMITED,FS1007,1,"10,294.00",Jan,2021,2
35300,28-Jan-21,DIAMOND TRUST BANK KENYA LIMITED,FS1007,1,"10,294.00",Jan,2021,2
35305,29-Jan-21,BRITISH AMERICAN TOBACCO KENYA PLC,LP-DF-FF0002,1,"2,983.00",Jan,2021,2


In [103]:
#Aggregating filters by year and item number
Fuelfilter['Quantity'] = pd.to_numeric(Fuelfilter['Quantity'].astype(int))
Avg_by_quater = Fuelfilter.loc[Fuelfilter['Quater']=='3'].groupby(['Item Number', 'Year']).sum()
Avg_by_quater

Quantity
Item Number  Year          
FF0563300    2020         2
FF0576400    2020         6
FF0579500    2020         1
FF105        2018        20
             2020         4
...                     ...
FS20088      2020         5
FS20203      2019       136
FS20402      2019        10
FS36230      2018         4
LP-DF-FF0002 2020         1

[194 rows x 1 columns]

In [107]:
df2 = pd.DataFrame(Avg_by_quater)
Fuelfilterdemand = df2.to_csv('FuelFilterDemand.csv')